In [1]:
import transformers

#Set to avoid warning messages.
transformers.logging.set_verbosity_error()

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 01.03. Using a Qu-An Pipeline

In [2]:
from transformers import pipeline

context="""
Earth is the third planet from the Sun and the only astronomical object 
known to harbor life. While large volumes of water can be found 
throughout the Solar System, only Earth sustains liquid surface water. 
About 71% of Earth's surface is made up of the ocean, dwarfing 
Earth's polar ice, lakes, and rivers. The remaining 29% of Earth's 
surface is land, consisting of continents and islands. 
Earth's surface layer is formed of several slowly moving tectonic plates, 
interacting to produce mountain ranges, volcanoes, and earthquakes. 
Earth's liquid outer core generates the magnetic field that shapes Earth's 
magnetosphere, deflecting destructive solar winds.
"""

quan_pipeline = pipeline("question-answering", 
                         model="deepset/minilm-uncased-squad2")

answer=quan_pipeline(question="How much of earth is land?",
             context=context)
print(answer)



2025-06-26 20:28:38.344181: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-26 20:28:38.847312: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-26 20:28:39.069281: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750969719.415372    7789 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750969719.526583    7789 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750969720.133349    7789 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

{'score': 0.9553402066230774, 'start': 327, 'end': 330, 'answer': '29%'}


In [3]:
print("\nAnother question :")
print(quan_pipeline( question="How are mountain ranges created?",
             context=context))


Another question :
{'score': 0.26154062151908875, 'start': 399, 'end': 471, 'answer': "Earth's surface layer is formed of several slowly moving tectonic plates"}


## 01.05 Evaluating Qu-An Performance

In [5]:
from evaluate import load
squad_metric = load("squad_v2")

#Ignoring Context & Question as they are not needed for evaluation
#This example is to showcase how the evaluation works based on match between the prediction
#and the correct answer

correct_answer="Paris"

predicted_answers=["Paris",
                 "London",
                 "Paris is one of the best cities in the world"]

cum_predictions=[]
cum_references=[]
for i in range(len(predicted_answers)):
    
    #Use the input format for predictions
    predictions = [{'prediction_text':predicted_answers[i], 
                    'id': str(i),
                    'no_answer_probability': 0.}]
    cum_predictions.append(predictions[0])
    
    #Use the input format for naswers
    references = [{'answers': {'answer_start': [1], 
                               'text': [correct_answer]}, 
                   'id': str(i)}]
    cum_references.append(references[0])

    results = squad_metric.compute(predictions=predictions,references=references)
    print("F1 is", results.get('f1'), " for answer :", predicted_answers[i])
    
#Compute for cumulative Results
cum_results=squad_metric.compute(predictions=cum_predictions,references=cum_references)
print("\n Cumulative Results : \n",cum_results)

F1 is 100.0  for answer : Paris
F1 is 0.0  for answer : London
F1 is 22.22222222222222  for answer : Paris is one of the best cities in the world

 Cumulative Results : 
 {'exact': 33.333333333333336, 'f1': 40.74074074074074, 'total': 3, 'HasAns_exact': 33.333333333333336, 'HasAns_f1': 40.74074074074074, 'HasAns_total': 3, 'best_exact': 33.333333333333336, 'best_exact_thresh': 0.0, 'best_f1': 40.74074074074074, 'best_f1_thresh': 0.0}
